In [ ]:
import nbformat

notebook_path = "Hierarchical Indices.ipynb"

nb = nbformat.read(notebook_path, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

nbformat.write(nb, notebook_path)

print("Fixed!")


In [1]:
!pip install langchain-community langchain-text-splitters  pypdf  langchain-huggingface sentence-transformers faiss-cpu ollama langchain-ollama   --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 72.1 MB/s eta 0:00:00


In [2]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.0 which is incompatible.


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
model_id = "Qwen/Qwen3-4B-Instruct-2507"

# ==========================================
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 確保 pipeline 正常運作的常見修復
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# ==========================================
# 建立原生 Transformers Pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,      # 決定回答長度
    temperature=0.2,         # RAG 建議設低一點，減少幻覺
    repetition_penalty=1.1,  # 避免鬼打牆
    return_full_text=False   # 重要：設為 False 只回傳生成的部分
)

# . 轉為 LangChain 的 LLM 物件
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# (關鍵) 轉為 ChatModel 物件
# ChatHuggingFace 會自動處理 prompt template (User/System/Assistant 格式)
chat_model = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'temperature', 'max_new_tokens', 'repetition_penalty'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cuda'},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [8]:
from langchain_community.document_loaders import PyPDFLoader
import asyncio
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from tqdm import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [9]:
path=r"/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf"
chunk_size=1024
chunk_overlap=128
query="What the CRAG proposed to improve "

以頁數為單位做summary

In [13]:
async def encode_pdf_hierarchical(path, chunk_size, chunk_overlap, is_string=False):
    #讀pdf
    loader=PyPDFLoader(path)
    documents= await asyncio.to_thread(loader.load) #[page1,page2...]
    #summary chain
    summary_template="""
    read the following context and write a brief summary:
    {context}
    summary:
    """
    summary_prompt=ChatPromptTemplate.from_template(summary_template)
    summary_chain= summary_prompt | chat_model | StrOutputParser()
    #creat document-level summaries
    async def summarized_doc(doc):
        summary_text=summary_chain.invoke({"context":doc.page_content})

        return Document(
                page_content=summary_text,
                metadata={
                    "source": path,
                    "page": doc.metadata["page"],
                    "summary": True
                    }
                )
    summaries=[]
    for doc in tqdm(documents):
        result=await summarized_doc(doc)
        if result:
            summaries.append(result)

    #creat detailed chunks
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    detailed_chunks=await asyncio.to_thread(text_splitter.split_documents,documents)

    for i,chunk in enumerate(detailed_chunks):
        chunk.metadata.update({
            "chunk_id":i,
            "summary":False,
            "page":int(chunk.metadata.get("pate",0)),
        })

    async def create_vectorstore(docs):
        return await asyncio.to_thread(FAISS.from_documents,docs,embedding_model)

    summary_vetorstore, detailed_vectorstore=await asyncio.gather(
        create_vectorstore(summaries),
        create_vectorstore(detailed_chunks)
    )
    return summary_vetorstore, detailed_vectorstore


建立FAISS Vectorstore

In [14]:
import os
summary_path = "/content/vector_stores/summary_store"
detailed_path = "/content/vector_stores/detailed_store"
if os.path.exists(summary_path) and os.path.exists(detailed_path):

    summary_store = FAISS.load_local(summary_path, embedding_model)
    detailed_store = FAISS.load_local(detailed_path, embedding_model)
else:
    summary_store, detailed_store = await encode_pdf_hierarchical(path,chunk_size,chunk_overlap)
    os.makedirs(summary_path, exist_ok=True)
    os.makedirs(detailed_path, exist_ok=True)
    summary_store.save_local(summary_path)
    detailed_store.save_local(detailed_path)

 62%|██████▎   | 10/16 [02:08<01:17, 12.97s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
100%|██████████| 16/16 [03:34<00:00, 13.43s/it]


先以頁為單位的summary做檢索,再以檢索到的頁數以chunk為單位做檢索

In [25]:
def retrieve_hierarchical(query, summary_vectorstore, detailed_vectorstore, k_summaries=2, k_chunks=5):
  top_summaries=summary_vectorstore.similarity_search(query,k=k_summaries)
  relevant_chunks=[]

  for summary in top_summaries:
    page_number=summary.metadata["page"]
    page_filter=lambda metadata: metadata["page"]==page_number
    page_chunks=detailed_vectorstore.similarity_search(query,k=k_chunks,filter=page_filter)
    relevant_chunks.extend(page_chunks)
  return relevant_chunks

In [26]:
results=retrieve_hierarchical(query,summary_store,detailed_store)
for chunk in results:
    print(f"Page: {chunk.metadata['page']}")
    print(f"Content: {chunk.page_content}...")
    print("---")

Page: 0
Content: knowledge retrieval actions discriminately. With
the further leverage of web search and optimized
knowledge utilization, CRAG has significantly
improved the ability of automatic self-correction
and efficient utilization of retrieved documents.
Experiments extensively demonstrate its adaptabil-
ity to RAG-based approaches as well as general-
izability across short- and long-form generation
tasks. While we primarily proposed to improve the
RAG framework from a corrective perspective and
CRAG can be seamlessly coupled with various
RAG-based approaches, fine-tuning an external
retrieval evaluator is inevitable. How to eliminate
this external evaluator and equip LLMs with better
retrieval evaluation capabilities will be our future
work.
References
Rohan Anil, Andrew M. Dai, Orhan Firat, Melvin John-
son, Dmitry Lepikhin, Alexandre Passos, Siamak
Shakeri, Emanuel Taropa, Paige Bailey, Zhifeng
Chen, Eric Chu, Jonathan H. Clark, Laurent El
Shafey, Yanping Huang, Kathy Meier-He